# Had to develop this blocks in another environment.
Please, uncomment the first import in the next block and delete the second block

In [21]:
#from ipynb.fs.defs.functions import train_test, classification_reporting

from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold

from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf

import pandas as pd
import numpy as np

In [22]:
# Delete this, if needed

from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize


# Define train/test dataset splitting, scaling implemented
def train_test(dataset, scaling = False):
    X = dataset.iloc[:,:dataset.shape[1]-1]
    y = dataset.iloc[:,-1]

    if scaling == True:
        X_columns = X.columns.to_list()
        X = pd.DataFrame(normalize(X, axis=0), columns=X_columns)

    X_train, X_test, y_train, y_test = train_test_split(X, y,
        test_size=0.2,
        stratify=dataset.iloc[:,-1],
        random_state=42)

    return X_train, X_test, y_train, y_test

# Define the classification report. "macro avg" for precision, recall and f1 will be used as a benchmark
def classification_reporting(tuning, model, y_test, y_pred, X_train):
    report = classification_report(y_test, y_pred, output_dict = True)
    print('{} {} performance using {} features:\n'.format(tuning, model, X_train.shape[1]),
          report['macro avg'])
    print('Accuracy score:', accuracy_score(y_test, y_pred))

In [23]:
# Split the dataset with 16 feature
dataset = pd.read_csv('/content/drive/MyDrive/dataset.csv').iloc[:,1:]
X_train_full, X_test_full, y_train_full, y_test_full = train_test(dataset, scaling = True)

y_train_full = pd.get_dummies(y_train_full)
y_test_full = pd.get_dummies(y_test_full)

In [24]:
# Define the model
def model():
  model = Sequential()

  # Dense layer, with 32 neurons, ReLu as benchmark
  model.add(Dense(32, input_dim=16, activation='relu'))

  # Turn off the 20% of neurons to avoid overfitting
  model.add(Dropout(0.2))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.2))

  # Final dense layer with 5 neurons, one for each label, softmax will output the probabilities of an item to belong to each class
  model.add(Dense(5, activation='softmax'))

  # Compile the model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

model = model()

# Fit the model with epochs = nobs * 3 and batch size = 32 (rules of thumb)
model.fit(X_train_full, y_train_full, epochs = 48, batch_size = 32, verbose = 0)

In [25]:
# Predict
y_pred = model.predict(X_test_full)

# One-hot encode the probabilities, so that the output is a 0/1 matrix. The class for an observation will be the index of the columns which has the highest probabilities among other columns
y_pred_matrix = pd.DataFrame(np.eye(5)[np.argmax(y_pred, axis = 1)])

classification_reporting('Naive', 'NN', y_test_full, y_pred_matrix, X_train_full)

63/63 [==============================] - 0s 1ms/step
Naive NN performance using 16 features:
 {'precision': 0.7370162743706152, 'recall': 0.7187045158140631, 'f1-score': 0.7210107864641808, 'support': 2000}
Accuracy score: 0.7185


In [26]:
accuracy = accuracy_score(y_test_full, y_pred.round())
print('Accuracy of predictions on the test set:', accuracy)

Accuracy of predictions on the test set: 0.6355
